<a href="https://colab.research.google.com/github/violeo-crypto/compling-hw/blob/main/naumova_%22w2v_hw_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [14]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [15]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

Размер словаря: 400000
Размерность векторов: 100


Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

In [16]:
import gensim.downloader as api

info = api.info()
print("Доступные категории:", list(info.keys()))
print("\n" + "="*50)

for category in info.keys():
    print(f"\nМодели в категории '{category}':")
    for model_name in info[category].keys():
        print(f"  - {model_name}")

Доступные категории: ['corpora', 'models']


Модели в категории 'corpora':
  - semeval-2016-2017-task3-subtaskBC
  - semeval-2016-2017-task3-subtaskA-unannotated
  - patent-2017
  - quora-duplicate-questions
  - wiki-english-20171001
  - text8
  - fake-news
  - 20-newsgroups
  - __testing_matrix-synopsis
  - __testing_multipart-matrix-synopsis

Модели в категории 'models':
  - fasttext-wiki-news-subwords-300
  - conceptnet-numberbatch-17-06-300
  - word2vec-ruscorpora-300
  - word2vec-google-news-300
  - glove-wiki-gigaword-50
  - glove-wiki-gigaword-100
  - glove-wiki-gigaword-200
  - glove-wiki-gigaword-300
  - glove-twitter-25
  - glove-twitter-50
  - glove-twitter-100
  - glove-twitter-200
  - __testing_word2vec-matrix-synopsis


corpora -- это текстовые корпуса, на которых обучаются модели из списка models.  

In [12]:
model = api.load("glove-twitter-100")

print(f"Размер словаря: {len(model.key_to_index)}")
print(f"Размерность векторов: {model.vector_size}")

Размер словаря: 1193514
Размерность векторов: 100


In [18]:
model = api.load("word2vec-ruscorpora-300")

print(f"Размер словаря: {len(model.key_to_index)}")
print(f"Размерность векторов: {model.vector_size}")

[==================================================] 100.0% 198.8/198.8MB downloaded
Размер словаря: 184973
Размерность векторов: 300


In [24]:
model = api.load("glove-wiki-gigaword-50")

print(f"Размер словаря: {len(model.key_to_index)}")
print(f"Размерность векторов: {model.vector_size}")

[==================================================] 100.0% 66.0/66.0MB downloaded
Размер словаря: 400000
Размерность векторов: 50


Датасет glove-twitter-100 обучен на твитах, содержит 1193514 слов, размерность векторов равна 100. В таком датасете содержатся хештеги и эмодзи, а также учитывается контекст неформального интернет общения.

Датасет word2vec-ruscorpora-300 состоит из текстов на русском языке (НКРЯ) имеет 184973 слов в словаре и охватыевает русский литературный язык. Такой датасет можно использовать для обработки русского языка.

Датасет glove-wiki-gigaword-50 состоит из 400000 слов и обучен на Wikipedia и Gigaword. Маленькая размерность векторов (50) позволяет быстро использовать такой датасет для экспериментов или обучения, т.к. с ним очень быстро работать.

**Базовые операции с векторами**

In [25]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [26]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

Слова, похожие на 'python':
  monty: 0.6886
  php: 0.5865
  perl: 0.5784
  cleese: 0.5447
  flipper: 0.5113


*Ваш ответ здесь*

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [27]:
model = api.load("word2vec-ruscorpora-300")

print(f"Размер словаря: {len(model.key_to_index)}")
print(f"Размерность векторов: {model.vector_size}")

Размер словаря: 184973
Размерность векторов: 300


2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [35]:
def find_similar_words(word):
    try:
        similar_words = model.most_similar(word, topn=10)
        return similar_words, score
    except:
        return f"Не удалось найти похожие слова для '{word}'"

result = find_similar_words(input("Введите любое слово:"))
print(result)

Введите любое слово:птица_NOUN
([('птичка_NOUN', 0.6903809905052185), ('пернатое_NOUN', 0.6857922673225403), ('ворона_NOUN', 0.6528410911560059), ('утка_NOUN', 0.6466763019561768), ('птичий_ADJ', 0.6466594934463501), ('зверь_NOUN', 0.6319288015365601), ('прилетный_ADJ', 0.6262426972389221), ('carpodacus_NOUN', 0.6246825456619263), ('гусь_NOUN', 0.6238204836845398), ('larus_NOUN', 0.6233668923377991)], 0.5112984776496887)


3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [36]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [37]:
model = Word2Vec(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    sg=1
)

In [38]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


4. Проверьте модель

In [39]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711


In [41]:
# Найдите слова, похожие на "духовка"
### ваш код здесь ###
try:
    similar = model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")
# Найдите слова, похожие на "овощи"
### ваш код здесь ###
try:
    similar = model.wv.most_similar('овощи', topn=5)
    print("Слова, похожие на 'овощи':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")

Слова, похожие на 'духовка':
  ингредиенты: 0.3199
  десерт: 0.3064
  холодильник: 0.2705
  питание: 0.2243
  пирог: 0.2142
Слова, похожие на 'овощи':
  мариновать: 0.2716
  хлеб: 0.2691
  гриль: 0.2546
  фольга: 0.2409
  сахар: 0.2108


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [43]:
ft_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)

6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [46]:
try:
    similar = ft_model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

try:
    similar = ft_model.wv.most_similar('духовка', topn=5)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

#слова, похожие на "овощи"
try:
    similar = ft_model.wv.most_similar('овощи', topn=5)
    print("Слова, похожие на 'овощи':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")

Слова, похожие на 'варить':
  жарить: 0.5353
  парить: 0.4805
  месить: 0.3541
  тушить: 0.3405
  специи: 0.2622
Слова, похожие на 'духовка':
  взбивать: 0.4565
  лимон: 0.3561
  салат: 0.3050
  курица: 0.3041
  тост: 0.2944
Слова, похожие на 'овощи':
  жарить: 0.2960
  фольга: 0.2574
  морковь: 0.2297
  соус: 0.2172
  торт: 0.2094


7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [50]:
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    # Word2Vec
    try:
        w2v_similar = model.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    # FastText
    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

# Сравниваем для разных слов
compare_models('learning')
compare_models('торелка')
compare_models('скаварада')
compare_models('булошка')


Сравнение для слова: 'learning'
  Word2Vec: слово не найдено
  FastText: ['духовка', 'пирог']

Сравнение для слова: 'торелка'
  Word2Vec: слово не найдено
  FastText: ['торт', 'месить']

Сравнение для слова: 'скаварада'
  Word2Vec: слово не найдено
  FastText: ['чай', 'фольга']

Сравнение для слова: 'булошка'
  Word2Vec: слово не найдено
  FastText: ['резать', 'чашка']


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [51]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']


In [52]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [53]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [54]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


8. Сравните схожесть doc_2 и doc_4

In [57]:
def compare_documents(doc2_id, doc4_id):
    similarity = doc_model.dv.similarity(f"doc_{doc2_id}", f"doc_{doc4_id}")
    print(f"Схожесть doc_{doc2_id} и doc_{doc4_id}: {similarity:.4f}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")
    print(f"  doc_{doc4_id}: {documents[doc4_id]}")

compare_documents(2, 4)  # machine learning vs deep learning


Схожесть doc_2 и doc_4: -0.0362
  doc_2: python programming for data science
  doc_4: computer vision processes images


9. Найдите самый похожий документ на doc_1

In [59]:

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_1", topn=2)
print("\nДокументы, похожие на doc_1:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")


Документы, похожие на doc_1:
  doc_0: 0.2735
    Текст: machine learning is interesting
  doc_3: 0.2031
    Текст: artificial intelligence is amazing


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [60]:
doc_model = FastText(
    sentences=cooking_sentences,
    vector_size=10,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

In [63]:
try:
    similar = doc_model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

try:
    similar = doc_model.wv.most_similar('торт', topn=5)
    print("Слова, похожие на 'торт':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'торт' не найдено в словаре")

try:
    similar = doc_model.wv.most_similar('кастрюля', topn=5)
    print("Слова, похожие на 'кастрюля':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'кастрюля' не найдено в словаре")

Слова, похожие на 'варить':
  готовить: 0.7548
  парить: 0.7298
  травы: 0.6633
  мука: 0.6609
  взбивать: 0.4333
Слова, похожие на 'торт':
  соус: 0.7231
  яичница: 0.7042
  холодильник: 0.6424
  лимон: 0.5706
  смешивать: 0.5632
Слова, похожие на 'кастрюля':
  барбекю: 0.5832
  сковорода: 0.5687
  месить: 0.5515
  сливки: 0.5105
  уголь: 0.4474


In [64]:
doc_model = FastText(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

In [65]:
try:
    similar = doc_model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

try:
    similar = doc_model.wv.most_similar('торт', topn=5)
    print("Слова, похожие на 'торт':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'торт' не найдено в словаре")

try:
    similar = doc_model.wv.most_similar('кастрюля', topn=5)
    print("Слова, похожие на 'кастрюля':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'кастрюля' не найдено в словаре")

Слова, похожие на 'варить':
  жарить: 0.5371
  парить: 0.4818
  месить: 0.3567
  тушить: 0.3435
  специи: 0.2624
Слова, похожие на 'торт':
  сахар: 0.4199
  печь: 0.3508
  кипятить: 0.2942
  овощи: 0.2093
  тушить: 0.1852
Слова, похожие на 'кастрюля':
  лимон: 0.3251
  запекать: 0.1585
  яйца: 0.1448
  хлеб: 0.1438
  ингредиенты: 0.1410


In [66]:
doc_model = FastText(
    sentences=cooking_sentences,
    vector_size=100,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

In [67]:
try:
    similar = doc_model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

try:
    similar = doc_model.wv.most_similar('торт', topn=5)
    print("Слова, похожие на 'торт':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'торт' не найдено в словаре")

try:
    similar = doc_model.wv.most_similar('кастрюля', topn=5)
    print("Слова, похожие на 'кастрюля':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'кастрюля' не найдено в словаре")

Слова, похожие на 'варить':
  жарить: 0.5905
  парить: 0.5618
  пирог: 0.2525
  готовить: 0.2489
  сахар: 0.2388
Слова, похожие на 'торт':
  мариновать: 0.2365
  соль: 0.1935
  вода: 0.1867
  травы: 0.1805
  сахар: 0.1713
Слова, похожие на 'кастрюля':
  яичница: 0.3326
  торт: 0.2024
  вино: 0.1752
  дрожжи: 0.1504
  соль: 0.1437
